# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

# Vamos dar seguimento a nossa analise pela quetão 7 (árvore de regressão)

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from patsy import dmatrices

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import Lasso,LinearRegression, Ridge 

from src.config import DADOS, PALETTE
from src.funcao import avaliar_modelo_r2, best_estimator



In [5]:
# Importando a base de dados
df = pd.read_csv(DADOS)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [7]:
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela
8. Avalie a previsão mediana do log(renda) nas bases de treinamento e teste. A previsão mediana do log(renda), por hipótese, é o mesmo que a previsão média do log(renda).
9. Obtenha a previsão mediana da renda - a mediana é o exponencial do item anterior. Observe que para a média não vale o mesmo resultado.
10. Divida a previsão do item 9 em 10 grupos. Calcule a média, mediana, percentil 5% e percentil 95% da renda para cada grupo. Faça um gráfico.
Analise e comente as limitações dessa previsão, e reflita sobre o quão útil é este modelo.









In [9]:
# convertendo a coluna "qtd_filhos" em float
df["qtd_filhos"] = df["qtd_filhos"].astype(float)



In [10]:
QUANTIL = 0.99

df_clean = df[
    (df["qtd_filhos"] < df["qtd_filhos"].quantile(QUANTIL)) &
    (df["tempo_emprego"] < df["tempo_emprego"].quantile(QUANTIL)) &
    (df["qt_pessoas_residencia"] < df["qt_pessoas_residencia"].quantile(QUANTIL)) &
    (df["renda"] < df["renda"].quantile(QUANTIL))  # Corrigindo o nome da coluna
]


In [11]:
# Remover valores nulo
df.dropna(axis=0,inplace=True)

# Removendo colunas desnecessárias
X = df.drop(["Unnamed: 0", "data_ref","id_cliente", "renda"], axis=1)
y = df["renda"]


In [12]:
# Transformando variáveis categórica em variáveis dummy
X = pd.get_dummies(X, drop_first=True)


In [13]:
y.info()

<class 'pandas.core.series.Series'>
Index: 12427 entries, 0 to 14999
Series name: renda
Non-Null Count  Dtype  
--------------  -----  
12427 non-null  float64
dtypes: float64(1)
memory usage: 194.2 KB


In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   posse_de_veiculo               12427 non-null  bool   
 1   posse_de_imovel                12427 non-null  bool   
 2   qtd_filhos                     12427 non-null  float64
 3   idade                          12427 non-null  int64  
 4   tempo_emprego                  12427 non-null  float64
 5   qt_pessoas_residencia          12427 non-null  float64
 6   sexo_M                         12427 non-null  bool   
 7   tipo_renda_Bolsista            12427 non-null  bool   
 8   tipo_renda_Empresário          12427 non-null  bool   
 9   tipo_renda_Pensionista         12427 non-null  bool   
 10  tipo_renda_Servidor público    12427 non-null  bool   
 11  educacao_Pós graduação         12427 non-null  bool   
 12  educacao_Secundário            12427 non-null  bool

In [15]:
# Separando em dados de treino (75%) e teste(25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [16]:
# Criar o modelo de árvore de regressão
tree_reg = DecisionTreeRegressor(max_depth=6, random_state=100)

In [17]:
# Treinar o modelo
tree_reg.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=6, random_state=100)

In [18]:
# Fazer previsões
y_pred_tree = tree_reg.predict(X_test)


In [19]:
# Calcular métricas
mse_tree = mean_squared_error(y_test, y_pred_tree)
r2_tree = r2_score(y_test, y_pred_tree)

print(f"Árvore de Regressão - MSE: {mse_tree:.4f}, R²: {r2_tree:.4f}")


Árvore de Regressão - MSE: 41361371.1503, R²: 0.3384


Até agora o modelo com maior $r^2$ é a árvore de regressão com o valor de 0.3384 .